In [133]:
import string
import re
import os
import nltk
import pandas as pd
import numpy as np
import json
import glob
import gzip
import tensorflow as tf
from tensorflow import keras
import time

import matplotlib.pyplot as plt

SEED = 1013
np.random.seed(SEED)

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


import pandas as pd 
import numpy as np 
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from scipy import sparse
import os
import pickle
import emoji
from wordcloud import WordCloud

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.util import ngrams

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer


from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Meghna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
import collections
df=pd.read_csv('/Users/Meghna/Desktop/tweets_for_language_model_noRT.csv')
df

,Unnamed: 0,races,tweets
0,0,0,And on the first day of the Colorado legislatu...
1,1,1,Everyone needs to read this. Dr. Nichols and h...
2,2,0,“He would make outrageous claims like he inven...
3,3,0,@holynext SUGAR AND COVID\nDiabetes is a targe...
4,4,1,I’m all for compromise - just do not give in o...
...,...,...,...
37199,37199,1,"There's great news about Covid vaccines, but w..."
37200,37200,1,@andybudd You're saying this like all travelli...
37201,37201,1,Oh they was beaten or pepper sprayed
37202,37202,1,Maine CDC prepares to distribute COVID-19 vacc...


In [10]:
races = df['races'].to_list()
count = 0
for val in races:
    if val == 1:
        count += 1
print(count)
df.head()

18602


,Unnamed: 0,races,tweets
0,0,0,And on the first day of the Colorado legislatu...
1,1,1,Everyone needs to read this. Dr. Nichols and h...
2,2,0,“He would make outrageous claims like he inven...
3,3,0,@holynext SUGAR AND COVID\nDiabetes is a targe...
4,4,1,I’m all for compromise - just do not give in o...


In [19]:
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+' #remove mentions
pat2 = r'https?://[^ ]+' #remove hyperlinks
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml') # remove html tags
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?") #remove byte order marks
    except:
        bom_removed = souped
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    bom_removed = emoji.demojize(bom_removed, language='en')    
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    stripped = re.sub(r'\@w+','',stripped)
    lower_case = stripped.lower()
    lower_case = emoji.demojize(lower_case, language='en')
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled) #remove hashtag
    tweets_clean = []
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    for word in words:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)
    return (" ".join(tweets_clean)).strip()

In [20]:
tweets = df['tweets'].to_list()
cleaned_tweets = []
for tweet in tweets:
    cleaned_tweets.append(tweet_cleaner_updated(tweet))

In [25]:
df_new = pd.DataFrame()
df_new['cleaned_tweets'] = cleaned_tweets
df_new['races'] = df['races']

In [46]:
!pip install --upgrade gensim

In [55]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import warnings

In [111]:
races = df['races'].to_list()

In [113]:
train_corpus,test_corpus,train_labels,test_labels = train_test_split(cleaned_tweets,races,stratify=races,test_size=0.25,random_state=1)

In [114]:
def strip_newline(series):
    return [review.replace('\n','') for review in series]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [115]:
def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

In [116]:
df_train = pd.DataFrame()
df_train['cleaned_tweets'] = train_corpus
df_train['races'] = train_labels

In [122]:
def get_corpus(df_train):
    """
    Get Bigram Model, Corpus, id2word mapping
    """
    
    df_train['cleaned_tweets'] = strip_newline(df_train.cleaned_tweets)
    words = list(sent_to_words(df_train.cleaned_tweets))
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
#     lemma = lemmatization(bigram)
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram

train_corpus2, train_id2word, bigram_train = get_corpus(df_train)

In [123]:
print(bigram_train[0])

['countri', 'led', 'women', 'cope', 'better', 'covid', 'via']


In [124]:
import logging # This allows for seeing if the model converges. A log file is created.
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [145]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus2,
                           num_topics=50,
                           id2word=train_id2word,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train.save('lda_train.model')
    

In [146]:
lda_train.print_topics(20,num_words=15)[:10]

[(15,
  '0.122*"hospit" + 0.049*"differ" + 0.042*"learn" + 0.029*"record" + 0.026*"fear" + 0.024*"women" + 0.023*"healthcar" + 0.022*"access" + 0.022*"took" + 0.022*"went" + 0.021*"among" + 0.020*"patient" + 0.019*"aid" + 0.017*"cancer" + 0.016*"huge"'),
 (43,
  '0.166*"right" + 0.112*"point" + 0.046*"backhand_index" + 0.045*"face_tear" + 0.035*"joy" + 0.033*"daili" + 0.025*"expos" + 0.021*"known" + 0.021*"review" + 0.020*"fauci" + 0.016*"joy_face" + 0.015*"tear_joy" + 0.015*"tori" + 0.014*"lie" + 0.014*"enforc"'),
 (11,
  '0.101*"wear_mask" + 0.063*"care" + 0.061*"pleas" + 0.037*"everyon" + 0.034*"someon" + 0.029*"reason" + 0.029*"other" + 0.029*"might" + 0.027*"children" + 0.027*"protect" + 0.025*"take" + 0.023*"need" + 0.023*"one" + 0.020*"power" + 0.018*"fold_hand"'),
 (42,
  '0.095*"countri" + 0.089*"world" + 0.044*"compani" + 0.042*"done" + 0.036*"someth" + 0.035*"heart" + 0.026*"would" + 0.024*"scientist" + 0.021*"name" + 0.021*"three" + 0.021*"smile_face" + 0.021*"pandem" + 0.0

In [157]:
train_vecs = []
for i in range(len(train_corpus)):
    top_topics = lda_train.get_document_topics(train_corpus2[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(50)]
    train_vecs.append(topic_vec)

In [158]:
train_vecs[2]

[0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.28856915,
 0.0028573493,
 0.14571416,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.14572206,
 0.0028573493,
 0.0028573493,
 0.1457059,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.0028573493,
 0.14570802,
 0.0028573493]

In [159]:
X = np.array(train_vecs)
y = np.array(train_labels)

In [160]:
kf = KFold(5, shuffle=True, random_state=42)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

for train_ind, val_ind in kf.split(X, y):
    # Assign CV IDX
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        fit_intercept=True
    ).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # Logistic Regression Mini-Batch SGD
    sgd = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # SGD Modified Huber
    sgd_huber = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20,
        loss='modified_huber',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average='binary'))

print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')

/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserW

Logistic Regression Val f1: 0.534 +- 0.004
Logisitic Regression SGD Val f1: 0.537 +- 0.023
SVM Huber Val f1: 0.399 +- 0.326
